In [2]:
import pandas as pd
import numpy as np
import json
import re
import os
from datetime import datetime
from typing import List, Dict, Optional, Tuple
from dataclasses import dataclass, asdict, field

import dspy
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

/home/atothegodd/living_lab/living_lab/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# ============================================================
# 1. DATA LOADING & MODEL CONFIG
# ============================================================

In [ ]:
students_df = pd.read_csv("students.csv")

# Configure LLM (Ollama or OpenAI)

# api_base  = 'https://uneuphonious-inductionless-arvilla.ngrok-free.dev'


# llm = dspy.LM(
#     model="ollama/gemma3:27b",
#     api_base=api_base,
#     temperature=0.0,
#     max_tokens=2000
# )

llm = dspy.LM(
    model="ollama/ministral-3:8b",
    temperature=0.0,
    max_tokens=2000
)


dspy.settings.configure(lm=llm)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Setup logging
RUN_ID = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
RUN_DIR = f"logs/run_{RUN_ID}"
os.makedirs(RUN_DIR, exist_ok=True)

MAX_PROJECTS = 5
print(f"✓ Setup complete. Run ID: {RUN_ID}")
print(f"✓ Students: {len(students_df)}")
# print(f"Projects: {len(projects_df)}")

✓ Setup complete. Run ID: 2026-01-22_17-11-33
✓ Students: 8


# ============================================================
# 2. CORE DATA STRUCTURES (STATE)
# ============================================================

In [ ]:
@dataclass
class ProjectRole:
    """Represents a single human role in a project."""
    role: str
    quota: int
    responsibility: str
    required_skills: List[str]

    def validate(self) -> bool:
        return (
            len(self.role.strip()) > 0
            and self.quota > 0
            and len(self.responsibility.strip()) > 0
            and 3 <= len(self.required_skills) <= 6
        )

@dataclass
class ProjectSpecification:
    """Complete, machine-consumable project specification."""
    project_summary: str
    project_type: List[str]
    headcount: int
    duration_months: int
    roles: List[ProjectRole]
    assumptions: List[str]
    risks: List[str]

    def validate(self) -> Dict:
        errors = []
        total_quota = sum(r.quota for r in self.roles)
        if total_quota != self.headcount:
            errors.append(f"Quota sum ({total_quota}) != headcount ({self.headcount})")
        for role in self.roles:
            if not role.validate():
                errors.append(f"Invalid role: {role.role}")
        if not self.project_type or len(self.project_type) == 0:
            errors.append("project_type cannot be empty")
        return {"is_valid": len(errors) == 0, "errors": errors}

    @staticmethod
    def to_dict(spec: 'ProjectSpecification') -> Dict:
        """Convert ProjectSpecification to dictionary."""
        return {
            "project_summary": spec.project_summary,
            "project_type": spec.project_type,
            "headcount": spec.headcount,
            "duration_months": spec.duration_months,
            "roles": [
                {
                    "role": r.role,
                    "quota": r.quota,
                    "responsibility": r.responsibility,
                    "required_skills": r.required_skills
                }
                for r in spec.roles
            ],
            "assumptions": spec.assumptions,
            "risks": spec.risks
        }

@dataclass
class ConversationState:
    messages: List[Dict] = field(default_factory=list)
    current_intent: Optional[str] = None
    extracted_project: Optional[ProjectSpecification] = None


@dataclass
class AgentState:
    students_df: Optional[pd.DataFrame] = None 
    project_spec: Optional[ProjectSpecification] = None
    project_row: Optional[pd.Series] = None  # optional / legacy
    ranked_df: Optional[Dict[str, pd.DataFrame]] = None
    decision_df: Optional[Dict[str, pd.DataFrame]] = None
    upskill_df: Optional[pd.DataFrame] = None
    explanations: Optional[List[Dict]] = None
    validations: Optional[pd.DataFrame] = None
    conversation: ConversationState = field(default_factory=ConversationState)
    flags: Dict[str, bool] = field(default_factory=dict)
    steps: int = 0
    done: bool = False
    
    
@dataclass
class AgentStateView:
    ranked: bool
    decided: bool
    explained: bool
    validated: bool
    needs_upskill: bool

## State Abstraction & Action Space

In [ ]:
def build_state_view(state: AgentState) -> AgentStateView:
    needs_upskill = False

    if isinstance(state.decision_df, dict):
        for df in state.decision_df.values():
            if df["decision"].isin(["UPSKILL", "REJECT"]).any():
                needs_upskill = True
                break

    return AgentStateView(
        ranked=state.ranked_df is not None,
        decided=state.decision_df is not None,
        explained=state.explanations is not None,
        validated=state.validations is not None,
        needs_upskill=needs_upskill,
    )


In [ ]:
ALLOWED_ACTIONS = [
    "RANK",
    "DECIDE",
    "PERSIST",
]

In [ ]:
def legal_actions(view: AgentStateView) -> List[str]:
    actions = []

    if not view.ranked:
        actions.append("RANK")
    elif not view.decided:
        actions.append("DECIDE")
    elif not view.explained:
        actions.append("EXPLAIN")
    else:
        actions.append("PERSIST")

    return actions

# 3. TOOL REGISTRY

In [ ]:
class ToolRegistry:
    def __init__(self):
        self.tools = {}

    def register(self, name: str, tool):
        self.tools[name] = tool

    def get(self, name: str):
        return self.tools[name]


## Tool 1

In [ ]:
ABBR_MAP = {
    "ml": "machine learning",
    "etl": "extract transform load",
    "cv": "computer vision",
    "nlp": "natural language processing",
    "dl": "deep learning",
}

def normalize_skill(text: str) -> str:
    text = text.lower().strip()
    tokens = re.split(r"[,\s]+", text)
    tokens = [ABBR_MAP.get(t, t) for t in tokens if t]
    return " ".join(tokens)


In [ ]:
class SkillEmbedderTool:
    def __init__(self, model):
        self.model = model
        self.cache: Dict[str, np.ndarray] = {}

    def embed_text(self, text: str) -> np.ndarray:
        text = normalize_skill(text)
        if text not in self.cache:
            self.cache[text] = self.model.encode(
                text,
                normalize_embeddings=True
            )
        return self.cache[text]

    def embed_batch(self, texts: List[str]) -> List[np.ndarray]:
        result = []
        for t in texts:
            result.append(self.embed_text(t))
        return result
    
skill_embedder = SkillEmbedderTool(embedding_model)

In [14]:
class SkillIndex:
    """
    Canonical skill index over skill_node.csv (10k skills)
    Responsibilities:
    - load + embed skill nodes ONCE
    - semantic search raw skill → canonical skills
    """

    def __init__(
        self,
        skill_csv_path: str,
        embedder: SkillEmbedderTool,
    ):
        self.embedder = embedder
        self.df = pd.read_csv(skill_csv_path)

        assert "skill_id" in self.df.columns
        assert "skill_name" in self.df.columns

        # normalize
        self.df["skill_name_norm"] = self.df["skill_name"].apply(normalize_skill)

        # embed once
        print("⏳ Embedding skill_node.csv ...")
        self.df["embedding"] = self.df["skill_name_norm"].apply(
            self.embedder.embed_text
        )

        self.emb_matrix = np.vstack(self.df["embedding"].values)  # (N, dim)

        print(f"✓ SkillIndex ready ({len(self.df)} skills)")

    def search(
        self,
        raw_skill: str,
        top_k: int = 5,
        min_sim: float = 0.5,
    ) -> List[Dict]:
        """
        Map raw / noisy skill text to canonical skills
        """
        q_emb = self.embedder.embed_text(raw_skill)

        sims = self.emb_matrix @ q_emb  # dot product (cosine)
        idx = np.argsort(-sims)[:top_k]

        results = []
        for i in idx:
            if sims[i] < min_sim:
                continue
            results.append({
                "skill_id": self.df.iloc[i]["skill_id"],
                "skill_name": self.df.iloc[i]["skill_name"],
                "score": float(sims[i]),
            })

        return results
    
skill_index = SkillIndex(
    skill_csv_path="skill_node.csv",
    embedder=skill_embedder
)

⏳ Embedding skill_node.csv ...
✓ SkillIndex ready (9734 skills)


In [ ]:
class StudentRankerTool:
    """
    Rank students per role using skill-level semantic matching + coverage
    """

    def rank_by_role(
        self,
        project_row: pd.Series,
        students_df: pd.DataFrame
    ) -> Dict[str, pd.DataFrame]:

        role_rankings = {}

        for role, required_skills in project_row["role_skill_map"].items():

            role_skill_embs = skill_embedder.embed_batch(required_skills)
            rows = []

            for _, student in students_df.iterrows():

                student_skills = [
                    normalize_skill(s)
                    for s in student["skills_text"].split(",")
                    if s.strip()
                ]

                student_skill_embs = skill_embedder.embed_batch(student_skills)

                sims = [
                    max(np.dot(s_emb, r_emb) for s_emb in student_skill_embs)
                    for r_emb in role_skill_embs
                ]

                score = max(sims)
                coverage = sum(s >= 0.6 for s in sims) / len(sims) if sims else 0.0

                rows.append({
                    "role": role,
                    "student_id": student["student_id"],
                    "student_name": student["name"],
                    "student_skills_text": student["skills_text"],
                    "score": round(score, 3),
                    "coverage": round(coverage, 2),
                    "current_assignments": student["current_assignments"],
                    "max_capacity": student["max_capacity"]
                })

            role_rankings[role] = (
                pd.DataFrame(rows)
                .sort_values("score", ascending=False)
                .reset_index(drop=True)
            )

        return role_rankings

student_ranker = StudentRankerTool()

## Tool 2

In [ ]:
class DecisionMakerTool:
    """
    TOOL 3: Makes matching decisions based on hard & soft constraints.
    Decisions: ACCEPT, UPSKILL, REJECT, WAITLIST
    Hard constraints:
      - Student capacity (current_assignments < max_capacity)
      - Minimum score threshold
    Soft constraints:
      - Coverage threshold (50% of skills)
    """
    
    def __init__(self, max_projects: int = 5):
        self.max_projects = max_projects
    
    def apply_constraints(self, match_row: pd.Series, project_row: pd.Series) -> Tuple[str, List[str]]:
        """
        Apply hard & soft constraints to a match.
        
        Returns:
            (decision, rationale_list)
        """
        min_score = float(project_row["min_score"])
        if match_row["score"] < min_score:
            return "REJECT", ["Semantic match score below threshold"]

        if match_row["coverage"] < 0.1:
            return "UPSKILL", ["Insufficient skill coverage"]

        return "ACCEPT", ["Score and coverage meet project requirements"]
    
    def progressive_match(
        self, 
        project_row: pd.Series, 
        ranked_df: pd.DataFrame, 
        initial_k: int = 5, 
        step_k: int = 3
    ) -> pd.DataFrame:
        """
        Progressive matching: expand search window until quota is filled.
        Prevents over-committing top candidates.
        
        Args:
            project_row: Project specification
            ranked_df: Pre-ranked students by score
            initial_k: Initial candidate window
            step_k: Expand window by this amount each iteration
        
        Returns:
            DataFrame with decisions for all evaluated students
        """
        quota = int(project_row["quota"])
        evaluated = []
        accepted_count = 0
        k = initial_k
        seen = set()

        while accepted_count < quota and k <= len(ranked_df):
            batch = ranked_df.iloc[:k]

            for _, row in batch.iterrows():
                if row["student_id"] in seen:
                    continue
                seen.add(row["student_id"])

                # Hard constraint: max projects capacity
                if row["current_assignments"] >= row["max_capacity"]:
                    decision, rationale = "REJECT", ["Student reached max project limit"]
                else:
                    decision, rationale = self.apply_constraints(row, project_row)

                evaluated.append({
                    **row.to_dict(),
                    "decision": decision,
                    "rationale": "; ".join(rationale)
                })

                if decision == "ACCEPT":
                    accepted_count += 1
                    if accepted_count == quota:
                        break

            k += step_k

        return pd.DataFrame(evaluated)
    
    def enforce_quota(self, decision_df: pd.DataFrame, project_row: pd.Series) -> pd.DataFrame:
        """
        After decisions made, move excess accepts to WAITLIST.
        Ensures exact quota match.
        """
        quota = int(project_row["quota"])

        accepted = (
            decision_df[decision_df["decision"] == "ACCEPT"]
            .sort_values("score", ascending=False)
            .head(quota)
        )

        waitlist = (
            decision_df[decision_df["decision"] == "ACCEPT"]
            .sort_values("score", ascending=False)
            .iloc[quota:]
        )

        final_df = decision_df.copy()
        final_df.loc[waitlist.index, "decision"] = "WAITLIST"
        final_df.loc[waitlist.index, "rationale"] = "Quota exceeded; added to waitlist"

        return final_df

decision_maker = DecisionMakerTool(max_projects=MAX_PROJECTS)

## Tool 3

In [ ]:
class ExplainDecision(dspy.Signature):
    """Explain why the matching agent made a decision for a student-project pair."""
    context = dspy.InputField()
    explanation = dspy.OutputField(desc="2-3 bullet points")

class DecisionExplainerTool(dspy.Module):
    """
    TOOL 4: Generates LLM-based explanations for non-accept decisions.
    Used to create human-readable rationales for UPSKILL/REJECT decisions.
    """
    
    def __init__(self):
        super().__init__()
        self.explain = dspy.ChainOfThought(ExplainDecision)
    
    def explain_decision(self, context: Dict) -> str:
        """
        Generate LLM explanation for a single decision.
        
        Args:
            context: Structured decision context
        
        Returns:
            LLM-generated explanation
        """
        result = self.explain(context=str(context))
        return result.explanation

explainer = DecisionExplainerTool()

## Tool 4

In [ ]:
class ValidateDecision(dspy.Signature):
    """Validate whether decision is consistent with scores and context."""
    context = dspy.InputField(desc="Structured decision context")
    verdict = dspy.OutputField(desc="One of: OK, SUSPICIOUS")
    comment = dspy.OutputField(desc="Short reason explaining verdict (1-2 sentences)")

class DecisionValidatorTool(dspy.Module):
    """
    TOOL 5: Validates matching decisions for consistency using LLM reasoning.
    Checks if REJECT/UPSKILL decisions are consistent with scores.
    """
    
    def __init__(self):
        super().__init__()
        self.validate = dspy.Predict(ValidateDecision)
    
    def validate_decision(self, context: Dict) -> Tuple[str, str]:
        """
        Validate a single decision.
        
        Returns:
            (verdict, comment) where verdict in {OK, SUSPICIOUS}
        """
        result = self.validate(context=str(context))
        return result.verdict, result.comment
    
    def validate_batch(self, decision_df: pd.DataFrame, project_row: pd.Series) -> pd.DataFrame:
        """Validate all non-ACCEPT decisions in a batch."""
        validation_results = []
        
        for _, row in decision_df.iterrows():
            if row["decision"] == "ACCEPT":
                continue
            context = self._build_context(row, project_row)
            verdict, comment = self.validate_decision(context)
            
            validation_results.append({
                "student_id": row["student_id"],
                "decision": row["decision"],
                "verdict": verdict,
                "comment": comment
            })
        
        return pd.DataFrame(validation_results)
    
    @staticmethod
    def _build_context(match_row: pd.Series, project_row: pd.Series, role: str) -> Dict:
        return {
            "project_title": project_row["title"],
            "project_description": project_row["description"],
            "project_role": role,
            "project_required_skills": ", ".join(project_row["role_skill_map"][role]),
            "student_name": match_row["student_name"],
            "student_skills": match_row["student_skills_text"],
            "semantic_score": round(match_row["score"], 3),
            "skill_coverage": round(match_row["coverage"], 2),
            "decision": match_row["decision"],
            "rule_rationale": match_row["rationale"],
        }

validator = DecisionValidatorTool()

## Tool 5

In [ ]:
class RecommendUpskill(dspy.Signature):
    """Recommend skill improvements based on student-project gap."""
    student_skills = dspy.InputField(desc="Current student skills")
    required_skills = dspy.InputField(desc="Project required skills")
    recommendation = dspy.OutputField(
        desc="Concrete skill gap analysis and learning recommendation (2-3 bullet points)"
    )

class UpskillCoachTool(dspy.Module):
    """
    TOOL 6: Generates personalized upskilling recommendations.
    For UPSKILL/REJECT decisions, proposes concrete learning path.
    """
    
    def __init__(self):
        super().__init__()
        self.recommend = dspy.ChainOfThought(RecommendUpskill)
    
    def generate_recommendation(self, student_skills: str, required_skills: str) -> str:
        """
        Generate upskill recommendation for a student.
        
        Args:
            student_skills: Current student skills
            required_skills: Project required skills
        
        Returns:
            LLM-generated recommendation
        """
        result = self.recommend(
            student_skills=student_skills,
            required_skills=required_skills
        )
        return result.recommendation
    
    def coach_batch(self, decision_df: pd.DataFrame, project_row: pd.Series) -> pd.DataFrame:
        """
        Generate recommendations for all UPSKILL/REJECT decisions.
        
        Args:
            decision_df: Decisions from DecisionMakerTool
            project_row: Project specification
        
        Returns:
            DataFrame with recommendations for non-accepted students
        """
        upskill_results = []

        for _, row in decision_df.iterrows():
            if row["decision"] not in ["UPSKILL", "REJECT"]:
                continue

            recommendation = self.generate_recommendation(
                student_skills=row["student_skills_text"],
                required_skills=project_row["required_skills_text"]
            )

            upskill_results.append({
                "student_id": row["student_id"],
                "student_name": row["student_name"],
                "decision": row["decision"],
                "recommendation": recommendation
            })

        return pd.DataFrame(upskill_results)

upskill_coach = UpskillCoachTool()

## Tool 6

In [ ]:
class ResultsPersisterTool:
    """
    TOOL 7: Persists all matching results and artifacts.
    Outputs: decisions.csv, accepted.csv, explanations.txt, upskill_plans.csv, summary.json
    """
    
    def __init__(self, run_dir: str):
        self.run_dir = run_dir
        os.makedirs(run_dir, exist_ok=True)
    
    def persist_decisions(self, decision_df: pd.DataFrame) -> str:
        path = f"{self.run_dir}/decisions.csv"
        decision_df.to_csv(path, index=False)
        return path
    
    def persist_accepted(self, decision_df: pd.DataFrame, project_row: pd.Series, run_id: str) -> str:
        """Save only accepted assignments."""
        accepted = decision_df[decision_df["decision"] == "ACCEPT"].copy()
        accepted["project_id"] = project_row["project_id"]
        accepted["project_title"] = project_row["title"]
        accepted["run_id"] = run_id
        
        cols = ["run_id", "project_id", "project_title", "student_id", "student_name", "score"]
        path = f"{self.run_dir}/accepted.csv"
        accepted[cols].to_csv(path, index=False)
        return path
    
    def persist_explanations(self, explanations: List[Dict]) -> str:
        """Save LLM-generated explanations."""
        path = f"{self.run_dir}/explanations.txt"
        with open(path, "w", encoding="utf-8") as f:
            for e in explanations:
                f.write(f"[{e['decision']}] Student {e['student_id']}\n")
                f.write(e["explanation"].strip() + "\n\n")
        return path
    
    def persist_validations(self, validation_df: pd.DataFrame) -> str:
        """Save validation results (SUSPICIOUS decisions only)."""
        suspicious = validation_df[validation_df["verdict"] == "SUSPICIOUS"]
        if suspicious.empty:
            return None
        
        path = f"{self.run_dir}/validation_alerts.txt"
        with open(path, "w", encoding="utf-8") as f:
            for _, row in suspicious.iterrows():
                f.write(
                    f"[SUSPICIOUS] Student {row['student_id']} "
                    f"(decision={row['decision']}): {row['comment']}\n"
                )
        return path
    
    def persist_upskill_plans(self, upskill_df: pd.DataFrame) -> str:
        """Save upskilling recommendations."""
        if upskill_df.empty:
            return None
        
        path = f"{self.run_dir}/upskill_plans.csv"
        upskill_df.to_csv(path, index=False)
        return path
    
    def persist_summary(self, project_row: pd.Series, decision_df: pd.DataFrame) -> str:
        summary = {
            "project_id": project_row["project_id"],
            "project_title": project_row["title"],
            "role_quotas": project_row["role_quota_map"],
            "accepted_count": int((decision_df["decision"] == "ACCEPT").sum()),
            "rejected_count": int((decision_df["decision"] == "REJECT").sum()),
            "upskill_count": int((decision_df["decision"] == "UPSKILL").sum()),
        }

        path = f"{self.run_dir}/summary.json"
        with open(path, "w", encoding="utf-8") as f:
            json.dump(summary, f, indent=2, ensure_ascii=False)
        return path


persister = ResultsPersisterTool(RUN_DIR)

# add tool in toolregistry

In [ ]:
tool_registry = ToolRegistry()
tool_registry.register("rank", student_ranker)
tool_registry.register("decide", decision_maker)
tool_registry.register("explain", explainer)
tool_registry.register("validate", validator)
tool_registry.register("upskill", upskill_coach)
tool_registry.register("persist", persister)

# Agent

## agent เลือก action ต่อไป

In [ ]:
class SelectNextAction(dspy.Signature):
    """
    Select the next agent action based on current abstract state.
    """
    state = dspy.InputField(desc="Current agent state (boolean flags)")
    allowed_actions = dspy.InputField(desc="List of allowed next actions")
    next_action = dspy.OutputField(
        desc="Choose exactly ONE action from allowed_actions"
    )


class ActionPolicy:
    def choose(self, state_view: AgentStateView, allowed: List[str]) -> str:
        return allowed[0]  # deterministic

## agent exec ทำตามคำสั่ง

In [ ]:
class ActionExecutor:

    def __init__(self, tools: ToolRegistry):
        self.tools = tools
    
    @staticmethod
    def filter_by_capacity(students_df: pd.DataFrame) -> pd.DataFrame:
        """Hard filter: only students with available capacity."""
        return students_df[
            students_df["current_assignments"] < students_df["max_capacity"]
        ].reset_index(drop=True)
    
    
    def execute(self, action: str, state: AgentState) -> AgentState:
        action = action.upper()


        if action == "RANK":
            filtered = self.filter_by_capacity(state.students_df)

            state.ranked_df = self.tools.get("rank").rank_by_role(
                state.project_row,
                filtered
            )


        elif action == "DECIDE":
            decisions = {}

            for role, ranked_df in state.ranked_df.items():
                quota = state.project_row["role_quota_map"][role]

                df = self.tools.get("decide").progressive_match(
                    project_row=pd.Series({
                        "quota": quota,
                        "min_score": state.project_row["min_score"]
                    }),
                    ranked_df=ranked_df
                )

                decisions[role] = df

            state.decision_df = decisions



        elif action == "EXPLAIN":
            # EXPLAIN: Generate LLM explanations for non-accept decisions
            explanations = []
            for role, df in state.decision_df.items():
                for _, row in df.iterrows():
                    if row["decision"] != "ACCEPT":
                        ctx = DecisionValidatorTool._build_context(row, state.project_row, role)
                        explanations.append({
                            "role": role,
                            "student_id": row["student_id"],
                            "decision": row["decision"],
                            "explanation": self.tools.get("explain").explain_decision(ctx)
                        })
                state.explanations = explanations



        elif action == "VALIDATE":
            # VALIDATE: Check decision consistency
            state.validations = self.tools.get("validate").validate_batch(
                state.decision_df, 
                state.project_row
            )


        elif action == "UPSKILL":
            # UPSKILL: Generate learning recommendations for rejected/upskill students
            state.upskill_df = self.tools.get("upskill").coach_batch(
                state.decision_df,
                state.project_row
            )


        elif action == "PERSIST":
            def flatten_decisions(decision_df_by_role: Dict[str, pd.DataFrame]) -> pd.DataFrame:
                frames = []
                for role, df in decision_df_by_role.items():
                    tmp = df.copy()
                    tmp["role"] = role
                    frames.append(tmp)
                return pd.concat(frames, ignore_index=True)
            # PERSIST: Save all artifacts
            persister = self.tools.get("persist")
            flat_df = flatten_decisions(state.decision_df)
            
            persister.persist_decisions(flat_df)
            persister.persist_accepted(flat_df, state.project_row, RUN_ID)
            persister.persist_summary(state.project_row, flat_df) 

            if state.explanations:
                persister.persist_explanations(state.explanations)

            if state.validations is not None and not state.validations.empty:
                persister.persist_validations(state.validations)

            if state.upskill_df is not None and not state.upskill_df.empty:
                persister.persist_upskill_plans(state.upskill_df)

        return state

## agent แปลง freetext เป็น structure ก่อนเข้าลูป

In [ ]:
class ExtractProjectSpec(dspy.Signature):
    """
    Extract structured project specification from ambiguous description.
    MUST satisfy all validation constraints.
    """
    project_description = dspy.InputField(desc="Unstructured project description")
    headcount = dspy.InputField(desc="Total number of people needed")
    duration = dspy.InputField(desc="Project duration (e.g., '3 months')")

    specification = dspy.OutputField(
        desc=(
            "Return ONLY valid JSON with fields:\n"
            "- project_summary (string)\n"
            "- project_type (non-empty list of strings)\n"
            "- headcount (int, must equal input)\n"
            "- duration_months (int)\n"
            "- roles (list of objects, MUST NOT be empty)\n"
            "   Each role MUST have:\n"
            "     - role (non-empty string)\n"
            "     - quota (int >= 1)\n"
            "     - responsibility (non-empty string)\n"
            "     - required_skills (list of 3 to 6 strings)\n"
            "   Sum of all role.quota MUST equal headcount\n"
            "- assumptions (list of strings)\n"
            "- risks (list of strings)\n\n"
            "If constraints cannot be met, ADJUST roles so that quota sum equals headcount.\n"
            "Do not include explanations. JSON only."
        )
    )

class ProjectExtractorTool(dspy.Module):
    def __init__(self, max_retries: int = 2):
        super().__init__()
        self.extractor = dspy.Predict(ExtractProjectSpec)
        self.max_retries = max_retries

    def forward(self, project_description: str, headcount: int, duration: str) -> ProjectSpecification:
        last_error = None

        for attempt in range(self.max_retries + 1):
            result = self.extractor(
                project_description=project_description,
                headcount=headcount,
                duration=duration
            )

            try:
                spec_json = self._parse_json(result.specification)
                spec = self._build_specification(spec_json, headcount)
                return spec
            except Exception as e:
                last_error = e

        raise ValueError(f"Project extraction failed after retries: {last_error}")

    
    @staticmethod
    def _parse_json(json_str: str) -> Dict:
        """Extract JSON object from LLM response."""
        cleaned = re.sub(r"```(?:json)?", "", json_str)
        cleaned = cleaned.strip()
        start = cleaned.find("{")
        if start == -1:
            raise ValueError("No JSON object found")
        stack = []
        end = None
        for i in range(start, len(cleaned)):
            if cleaned[i] == "{":
                stack.append("{")
            elif cleaned[i] == "}":
                stack.pop()
                if not stack:
                    end = i + 1
                    break
        if end is None:
            raise ValueError("Unbalanced JSON braces")
        json_candidate = cleaned[start:end]
        try:
            return json.loads(json_candidate)
        except json.JSONDecodeError as e:
            raise ValueError(f"Failed to parse JSON: {e}")
    
    @staticmethod
    def _build_specification(spec_dict: Dict, headcount: int) -> ProjectSpecification:
        roles = []

        for role_dict in spec_dict.get("roles", []):
            required_skills = role_dict.get("required_skills", [])
            if isinstance(required_skills, str):
                required_skills = [s.strip() for s in required_skills.split(",") if s.strip()]

            role = ProjectRole(
                role=str(role_dict.get("role", "")).strip().upper(),
                quota=int(role_dict.get("quota", 0)),
                responsibility=str(role_dict.get("responsibility", "")).strip(),
                required_skills=required_skills
            )
            roles.append(role)

        spec = ProjectSpecification(
            project_summary=str(spec_dict.get("project_summary", "")).strip(),
            project_type=spec_dict.get("project_type", []),
            headcount=headcount,
            duration_months=int(spec_dict.get("duration_months", 0)),
            roles=roles,
            assumptions=spec_dict.get("assumptions", []),
            risks=spec_dict.get("risks", [])
        )

        validation = spec.validate()
        if not validation["is_valid"]:
            raise ValueError(f"Specification validation failed: {validation['errors']}")

        return spec

#### ============================================================
### 8. AUTONOMOUS AGENT (DETERMINISTIC ORCHESTRATION)
#### The agent is the main loop that:
#### 1. Receives a ProjectSpecification and list of students
#### 2. Delegates to tools via an executor
#### 3. Follows a rule-based planner for deterministic ordering
#### 4. Returns final matching decisions
#### ============================================================


In [ ]:
def project_spec_to_row(spec: ProjectSpecification) -> pd.Series:
    role_skill_map = {}

    for r in spec.roles:
        canonical_skills = []

        for raw_skill in r.required_skills:
            hits = skill_index.search(raw_skill, top_k=3)

            # เก็บชื่อ canonical skill
            canonical_skills.extend(
                h["skill_name"].lower() for h in hits
            )

        # dedup
        role_skill_map[r.role.lower()] = list(set(canonical_skills))

    role_quota_map = {
        r.role.lower(): r.quota
        for r in spec.roles
    }

    return pd.Series({
        "project_id": "P_CHAT",
        "title": spec.project_summary[:50],
        "description": spec.project_summary,
        "role_skill_map": role_skill_map,     # 🔥 canonical now
        "role_quota_map": role_quota_map,
        "min_score": 0.35
    })


In [ ]:
def enforce_legality(proposed: str, allowed: List[str]) -> str:
    """
    Ensure proposed action is legal.
    Fallback to first allowed action if illegal.
    """
    if proposed in allowed:
        return proposed
    return allowed[0]


In [ ]:
class AutonomousMatchingAgent:
    def __init__(self, policy, executor):
        self.executor = executor
        self.policy = policy

    def run(self, project_spec, students_df):
        project_row = project_spec_to_row(project_spec)

        state = AgentState(
            project_spec=project_spec,
            project_row=project_row,
            students_df=students_df
        )

        max_steps = 10
        while not state.done and state.steps < max_steps:
            state.steps += 1

            view = build_state_view(state)
            allowed = legal_actions(view)
            proposed = self.policy.choose(view, allowed)
            action = enforce_legality(proposed, allowed)

            state = self.executor.execute(action, state)

            if action == "PERSIST":
                state.done = True

        return state


# interaction

In [ ]:
class ClassifyIntent(dspy.Signature):
    user_message = dspy.InputField()
    intent = dspy.OutputField(
        desc="One of: CREATE_PROJECT, RUN_MATCHING, ASK_EXPLANATION, MODIFY_CONSTRAINT, CHAT"
    )

class IntentRouter(dspy.Module):
    def __init__(self):
        self.classifier = dspy.Predict(ClassifyIntent)
        
    def route(self, text: str, state: AgentState) -> str:
        text_l = text.lower()

        if state.flags.get("matched") and text_l.startswith("ทำไม"):
            return "ASK_EXPLANATION"

        if state.flags.get("project_ready") and any(k in text_l for k in ["จัดทีม", "assign", "match", "เริ่ม", "run"]):
            return "RUN_MATCHING"

        if any(k in text_l for k in ["โปรเจกต์", "project", "ทำระบบ", "อยากได้"]):
            return "CREATE_PROJECT"

        # fallback to LLM
        return self.classifier(user_message=text).intent


In [ ]:
def extract_student_name(text: str) -> str:
    """
    very simple heuristic:
    'ทำไม สมชาย ไม่ผ่าน' -> 'สมชาย'
    """
    text = text.replace("ทำไม", "").replace("ไม่ผ่าน", "")
    text = text.replace("เพราะอะไร", "")
    return text.strip()


In [ ]:
def find_student(decision_df_by_role, name):
    for role, df in decision_df_by_role.items():
        match = df[df["student_name"].str.contains(name, case=False)]
        if not match.empty:
            return match.iloc[0], role
    return None, None


In [ ]:
def summarize_by_student(decision_df_by_role: Dict[str, pd.DataFrame]) -> pd.DataFrame:
    # flatten role -> rows
    flat = pd.concat(
        df.assign(role=role)
        for role, df in decision_df_by_role.items()
    )

    # priority: ACCEPT > UPSKILL > REJECT
    priority = {"ACCEPT": 3, "UPSKILL": 2, "REJECT": 1}

    final = (
        flat.assign(p=flat["decision"].map(priority))
            .sort_values("p", ascending=False)
            .groupby("student_id", as_index=False)
            .first()
    )

    return final


In [ ]:
def chat_loop(agent: AutonomousMatchingAgent, extractor: ProjectExtractorTool):

    state = AgentState(students_df=students_df)
    router = IntentRouter()

    print("🤖 สวัสดีครับ ผมช่วยจัดทีมให้โปรเจกต์ได้ 😊")

    while True:
        user_text = input("👤 ")
        if user_text.lower() in ["exit", "quit"]:
            break

        intent = router.route(user_text, state)

        # --- Free chat (clarification loop) ---
        if intent == "CHAT":
            print("🤖 เล่าโปรเจกต์คร่าว ๆ ได้เลยครับ เดี๋ยวผมจัดการให้")

        # --- User describes project and we extract it ---
        elif intent == "CREATE_PROJECT":
            print("🤖 โอเคครับ ผมสรุปโปรเจกต์ให้นะ")
            try:
                spec = extractor(
                    project_description=user_text,
                    headcount=3,
                    duration="3 months"
                )
                
                state.conversation.extracted_project = spec
                state.flags["project_ready"] = True

                print("🤖 ผมเข้าใจว่าโปรเจกต์คือ:")
                print(json.dumps(ProjectSpecification.to_dict(spec), indent=2, ensure_ascii=False))

                print("🤖 ถ้าโอเค พิมพ์ว่า 'จัดทีมเลย' ได้ครับ")
            except Exception as e:
                print(f"🤖 ขอโทษนะครับ ผมสรุปไม่ได้ ({e})")

        # --- Trigger autonomous matching ---
        elif intent == "RUN_MATCHING":
            if not state.conversation.extracted_project:
                print("🤖 ขอรายละเอียดโปรเจกต์ก่อนนะครับ")
                continue

            print("🤖 กำลังจัดทีมให้ครับ ⏳")

            final_state = agent.run(
                project_spec=state.conversation.extracted_project,
                students_df=students_df
            )

            # ✅ เก็บ state หลัง run เสร็จ
            state.flags["matched"] = True
            state.decision_df = final_state.decision_df
            state.project_row = final_state.project_row

            final_students = summarize_by_student(state.decision_df)

            accepted = (final_students["decision"] == "ACCEPT").sum()
            rejected = (final_students["decision"] == "REJECT").sum()
            upskill  = (final_students["decision"] == "UPSKILL").sum()


            print("🤖 เสร็จแล้วครับ 🎉")
            print(f"✓ ยอมรับ: {accepted}")
            print(f"✓ ปฏิเสธ: {rejected}")
            print(f"✓ อบรมเพิ่มเติม: {upskill}")
            print("🤖 ถ้าอยากรู้เหตุผลของใคร พิมพ์ว่า 'ทำไม <ชื่อ>' ได้เลยครับ")
            
            
        elif intent == "ASK_EXPLANATION":
            if not state.flags.get("matched"):
                print("🤖 ยังไม่ได้จัดทีมเลยครับ")
                continue

            name = extract_student_name(user_text)
            row, role = find_student(state.decision_df, name)

            if row is None:
                print("🤖 ผมหาคนนั้นไม่เจอครับ")
                continue

            ctx = DecisionValidatorTool._build_context(
                match_row=row,
                project_row=state.project_row,
                role=role
            )

            explanation = explainer.explain_decision(ctx)

            print(f"🤖 เหตุผลสำหรับ {row['student_name']}:")
            print(explanation)

        else:
            print("🤖 ผมยังไม่แน่ใจ ลองอธิบายโปรเจกต์เพิ่มอีกนิดได้ไหมครับ")

# Test

In [119]:
executor = ActionExecutor(tool_registry)
policy = ActionPolicy()

agent = AutonomousMatchingAgent(
    policy=policy,
    executor=executor
)

print("✓ Planner, Executor, and Agent initialized.")

✓ Planner, Executor, and Agent initialized.


In [120]:
extractor = ProjectExtractorTool()
chat_loop(agent, extractor)

🤖 สวัสดีครับ ผมช่วยจัดทีมให้โปรเจกต์ได้ 😊
🤖 โอเคครับ ผมสรุปโปรเจกต์ให้นะ
🤖 ผมเข้าใจว่าโปรเจกต์คือ:
{
  "project_summary": "พัฒนาและติดตั้ง model prediction เพื่อการใช้งานในการทำนายผลลัพธ์ทางธุรกิจหรือเทคโนโลยี โดยรวมถึงการเตรียมข้อมูล การฝึก model การทดสอบ และการ deploy model ในระบบการผลิต",
  "project_type": [
    "Machine Learning",
    "Data Science",
    "Software Development"
  ],
  "headcount": 3,
  "duration_months": 3,
  "roles": [
    {
      "role": "DATA SCIENTIST",
      "quota": 1,
      "responsibility": "เตรียมข้อมูล การวิเคราะห์ข้อมูล และการฝึก model prediction",
      "required_skills": [
        "Python (Pandas, Scikit-learn, TensorFlow/PyTorch)",
        "การวิเคราะห์ข้อมูลและสถิติ",
        "การจัดการข้อมูล (ETL)",
        "การประมวลผลข้อมูลใหญ่ (Big Data) - เตรียมสำหรับข้อมูลขนาดใหญ่"
      ]
    },
    {
      "role": "MACHINE LEARNING ENGINEER",
      "quota": 1,
      "responsibility": "การพัฒนาและปรับปรุง model prediction การทดสอบ model และการ optimize performa

KeyboardInterrupt: Interrupted by user